In [10]:
import pprint
import datetime
import pyaurorax
import cartopy.crs
import numpy as np
from concurrent.futures import ProcessPoolExecutor

aurorax = pyaurorax.PyAuroraX()
at = aurorax.tools

# Create TREx RGB mosaic movie

Using PyAuroraX's built-in movie function in combination with the mosaic tools, we can generate a mosaic movie.

## Step 1: Download and read image data

Pay attention to the order of the sites here is very important, as that order MUST be the same for both the skymap and image data we pass into the mosaic prep routines.

In [11]:
# For this example, we will just make a movie for 3 minutes of TREx RGB
dataset_name = "TREX_RGB_RAW_NOMINAL"
start_dt = datetime.datetime(2023, 2, 24, 6, 15)
end_dt = datetime.datetime(2023, 2, 24, 6, 17)
site_uid_list = ['yknf', 'gill', 'rabb']
data_download_objs = {}
for site_uid in site_uid_list:
    download_obj = aurorax.data.ucalgary.download(dataset_name, start_dt, end_dt, site_uid=site_uid)
    data_download_objs[site_uid] = download_obj

# Read in the data site-by-site, as we need this separation for mosaicing
data_list = []
for site_uid, download_obj in data_download_objs.items():
    data_list.append(aurorax.data.ucalgary.read(download_obj.dataset, download_obj.filenames))

## Step 2: Download and read skymaps

In [12]:
# download and read skymaps
skymaps = []
for site_uid in site_uid_list:
    download_obj = aurorax.data.ucalgary.download_best_skymap("TREX_RGB_SKYMAP_IDLSAV", site_uid, start_dt)
    skymap = aurorax.data.ucalgary.read(download_obj.dataset, download_obj.filenames[-1])
    skymaps.append(skymap.data[0])

pprint.pprint(skymaps)

[Skymap(project_uid=rgb, site_uid=yknf, imager_uid=rgb-08, site_map_latitude=62.519848, site_map_longitude=245.686966, ...),
 Skymap(project_uid=rgb, site_uid=gill, imager_uid=rgb-04, site_map_latitude=56.376724, site_map_longitude=265.356323, ...),
 Skymap(project_uid=rgb, site_uid=rabb, imager_uid=rgb-06, site_map_latitude=58.227810, site_map_longitude=256.319366, ...)]


## Step 3: Prep skymap and image data

In [13]:
# if we're not sure which altitudes are pre-computed, we can see them inside a skymap file
#
# if you choose different altitude when preparing the skymap data, the function will take longer
# to process as it performs an interpolation between the pre-computed altitudes
print("Available pre-computed altitudes (km): %s" % (', '.join(["%d" % (x) for x in skymaps[0].get_precalculated_altitudes()])))

Available pre-computed altitudes (km): 90, 110, 150


In [14]:
# prepare the skymap data
#
# NOTE: this step is not time dependent, so it only needs to be performed once
# per set of skymaps.
prepped_skymaps = at.mosaic.prep_skymaps(skymaps, 110, n_parallel=5)
print(prepped_skymaps)

Preparing skymaps:   0%|          | 0/3 [00:00<?, ?skymap/s]

MosaicSkymap(polyfill_lat=array(dims=(5, 265440), dtype=float64), polyfill_lon=array(dims=(5, 265440), dtype=float64), elevation=array(dims=(265440,), dtype=float32), site_uid_list=['yknf', 'gill', 'rabb'])


In [15]:
# prepare the image data
prepped_images = at.mosaic.prep_images(data_list)

## Step 4: Define mosaic parameters

In [16]:
# define the intensity scales for each site
#
# NOTE: you can define intensity scale for each site separately, all sites as a whole, or not at all
scale = {
    "yknf": (10, 105),
    "gill": (10, 105),
    "rabb": (10, 105),
}

# create projection
center_lat = 55.0
center_lon = -100.0
projection_obj = cartopy.crs.NearsidePerspective(central_latitude=center_lat, central_longitude=center_lon)
map_extent = [-145, -65, 35, 80]

## Step 5: Prepare mosaic frames

For each timestamp of interest, we will need to:

1. Create the mosaic using `at.mosaic.create()`
2. Plot the mosaic using the `Mosaic.plot()` method to obtain the figure
3. Save that figure as a frame, for use by the `at.movie()` function

Since there's a lot of frames and this task is very parallel-friendly, we'll leverage Python's `multiprocessing` and `tqdm` libraries to help us speed things up.

In [17]:
import os
import platform
import matplotlib.pyplot as plt
from tqdm.contrib.concurrent import process_map as tqdm_process_map
from tqdm import tqdm

# Let's make one mosaic frame for each timestamp available in the prepared data
mosaic_dt_list = prepped_images.timestamps


def process_frame(mosaic_dt):
    # Create the mosaic for each timestamp
    mosaic = at.mosaic.create(prepped_images, prepped_skymaps, mosaic_dt, projection_obj, image_intensity_scales=scale)

    # Plot this mosaic
    fig, ax = mosaic.plot(map_extent, title="TREx RGB - %s" % (mosaic_dt.strftime("%Y-%m-%d %H:%M:%S")), returnfig=True)

    # Save this mosaic frame
    filename = "movie_frames/%s_trex_rgb_mosaic.png" % (mosaic_dt.strftime("%Y%m%d_%H%M%S"))
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    plt.savefig(filename, bbox_inches="tight")
    plt.close()
    return filename


if (platform.system() == "Windows"):
    # pre-process frames serially
    #
    # NOTE: multiprocessing on Windows from within a notebook is not too easy, so we'll
    # just do this serially.
    frame_filename_list = []
    for i in tqdm(mosaic_dt_list, total=len(mosaic_dt_list), desc="Generating frame files: ", unit="frames"):
        frame_filename_list.append(process_frame(i))
else:
    frame_filename_list = tqdm_process_map(
        process_frame,
        mosaic_dt_list,
        max_workers=5,
        chunksize=1,
        desc="Generating frame files: ",
        unit="frames",
    )

Generating frame files:   0%|          | 0/60 [00:00<?, ?frames/s]

Now that we have our frames saved to disk, we'll use the `movie()` function to create the movie.

In [18]:
# generate the movie
at.movie(frame_filename_list, "test_trex_rgb_mosaic_movie.mp4", n_parallel=5, fps=10)

Reading files:   0%|          | 0/60 [00:00<?, ?files/s]

Encoding frames:   0%|          | 0/60 [00:00<?, ?frames/s]